In [2]:
%run "imports.ipynb"

In [3]:
tbl_samples = pd.read_csv('samples.meta.txt', sep="\t")
tbl_samples.head()

,ox_code,src_code,population,country,location,site,contributor,contact,year,m_s,sex,n_sequences,mean_coverage,ebi_sample_acc,latitude,longitude
0,AA0040-C,Twifo_Praso__E2,GHcol,Ghana,Twifo Praso,Twifo Praso,David Weetman,David Weetman,2012,M,F,95033368,30.99,ERS311878,5.60858,-1.54926
1,AA0041-C,Twifo_Praso__H3,GHcol,Ghana,Twifo Praso,Twifo Praso,David Weetman,David Weetman,2012,M,F,95843804,31.70,ERS311886,5.60858,-1.54926
2,AA0042-C,Takoradi_C7,GHcol,Ghana,Takoradi,Takoradi,David Weetman,David Weetman,2012,M,F,107420666,35.65,ERS311894,4.91217,-1.77397
3,AA0043-C,Takoradi_H8,GHcol,Ghana,Takoradi,Takoradi,David Weetman,David Weetman,2012,M,F,95993752,29.46,ERS311902,4.91217,-1.77397
4,AA0044-C,Takoradi_D10,GHcol,Ghana,Takoradi,Takoradi,David Weetman,David Weetman,2012,M,F,103044262,33.67,ERS311910,4.91217,-1.77397


In [3]:
calldata_hap_phase2.tree()

/
 ├── 2L
 │   ├── calldata
 │   │   └── GT (8906423, 1164, 2) int8
 │   ├── samples (1164,) |S8
 │   └── variants
 │       ├── ALT (8906423,) |S1
 │       ├── POS (8906423,) int32
 │       └── REF (8906423,) |S1
 ├── 2R
 │   ├── calldata
 │   │   └── GT (12047846, 1164, 2) int8
 │   ├── samples (1164,) |S8
 │   └── variants
 │       ├── ALT (12047846,) |S1
 │       ├── POS (12047846,) int32
 │       └── REF (12047846,) |S1
 ├── 3L
 │   ├── calldata
 │   │   └── GT (7897666, 1164, 2) int8
 │   ├── samples (1164,) |S8
 │   └── variants
 │       ├── ALT (7897666,) |S1
 │       ├── POS (7897666,) int32
 │       └── REF (7897666,) |S1
 ├── 3R
 │   ├── calldata
 │   │   └── GT (10752701, 1164, 2) int8
 │   ├── samples (1164,) |S8
 │   └── variants
 │       ├── ALT (10752701,) |S1
 │       ├── POS (10752701,) int32
 │       └── REF (10752701,) |S1
 └── X
     ├── calldata
     │   └── GT (4472265, 1099, 2) int8
     ├── samples (1099,) |S8
     └── variants
         ├── ALT (4472265,) |S1
         ├── POS (4472265,) int32
         └── REF (4472265,) |S1

In [4]:
allel.SortedIndex(calldata_hap_phase2['3L']['variants']['POS'][:])

0,1,2,3,4,...,7897661,7897662,7897663,7897664,7897665
9790,9791,9798,9812,9815,...,41956530,41956532,41956537,41956541,41956551


In [5]:
fasta_fn = 'data/Anopheles-gambiae-PEST_CHROMOSOMES_AgamP3.fa'
genome = pyfasta.Fasta(fasta_fn)

In [6]:
sorted(genome.keys())

['2L', '2R', '3L', '3R', 'UNKN', 'X', 'Y_unplaced']

-----------------------------------------------------

## DXY pairwise calc

In [7]:
import sys
import gc
import datetime
import humanize
from humanize import naturalsize, intcomma, intword


def log(*msg):
    print(' '.join(map(str, msg)), file=sys.stdout)
    sys.stdout.flush()
    
from contextlib import contextmanager

@contextmanager
def timer(*msg):
    before = datetime.datetime.now()
    try:
        yield
    except:
        after = datetime.datetime.now()
        elapsed = (after - before).total_seconds()
        done = 'errored after %s' % humanize.naturaldelta(elapsed)
        if not msg:
            msg = done
        else:
            msg = ', '.join(map(str, msg)) + ', ' + done
        print(msg, file=sys.stderr)
        sys.stderr.flush()   
        raise
    else:
        after = datetime.datetime.now()
        elapsed = (after - before).total_seconds()
        done = 'done in %s' % humanize.naturaldelta(elapsed)
        if not msg:
            msg = done
        else:
            msg = ', '.join(map(str, msg)) + ', ' + done
        print(msg, file=sys.stdout)
        sys.stdout.flush()

In [8]:
dist_dn_template = os.path.join('/bucket/dist_haps/')
dist_fn_template = '{chrom}.{start:08d}.{stop:08d}.npy'

In [54]:
def compute_dxy_distance_matrices(chrom, window_size):
    dist_dn = dist_dn_template.format(metric='dxy', window_size=window_size)
    if not os.path.exists(dist_dn):
        os.makedirs(dist_dn)
        
    
    # load accessibility map
    is_accessible = accessibility[chrom]['is_accessible'][:]
    
    # determine accessible positions
    pos_accessible, = np.nonzero(is_accessible)
    
    # define equally accessible windows
    window_starts = pos_accessible[0:None:window_size]
    window_stops = pos_accessible[window_size-1:None:window_size]
    
    # add final window to end of chromosome
    window_starts = np.append(window_starts, [window_stops[-1] + 1])
    window_stops = np.append(window_stops, [len(genome[chrom])])
    
    # load variant positions
    pos = allel.SortedIndex(calldata_hap_phase2[chrom]['variants']['POS'][:])

    # iterate over windows
    for window_start, window_stop in zip(window_starts, window_stops):
        
        # distance matrix file name
        dist_fn = dist_fn_template.format(chrom=chrom, start=window_start, stop=window_stop)
        dist_path = os.path.join(dist_dn, dist_fn)
        
        # stay dry
        if os.path.exists(dist_path):
            log('skipping', dist_path)
            
        else:
            log('building', dist_path)
            gc.collect()
            
            with timer():
                
                # locate the window
                loc = pos.locate_range(window_start, window_stop)
                print (loc.start, loc.stop)

                # load data
                genotypes_phase2_call = calldata_hap_phase2[chrom]["calldata/GT"]
                genotypes_phase2 = allel.GenotypeChunkedArray(genotypes_phase2_call[loc])
                haplotypes = genotypes_phase2.to_haplotypes()
                n_variants = genotypes_phase2.shape[0]
                log('variants:', n_variants)


                # compute hamming distance
                dist = allel.pairwise_distance(haplotypes[:], metric='hamming')
                log('hamming distance, max:', dist.max(), ', min:', dist.min())

                # adjust by accessible window size
                n_bases = np.count_nonzero(is_accessible[window_start:window_stop+1])
                log('window accessible size:', n_bases)
                dist = dist * n_variants / n_bases
                log('dxy distance, max:', dist.max(), ', min:', dist.min())

                # save
                np.save(dist_path, dist)

In [55]:
compute_dxy_distance_matrices('3L', 100000)

skipping /bucket/dist_haps/3L.00009778.00730787.npy
skipping /bucket/dist_haps/3L.00730788.01243275.npy
skipping /bucket/dist_haps/3L.01243276.01622713.npy
skipping /bucket/dist_haps/3L.01622714.01833911.npy
skipping /bucket/dist_haps/3L.01833912.01989789.npy
skipping /bucket/dist_haps/3L.01989790.02108439.npy
skipping /bucket/dist_haps/3L.02108440.02239834.npy
skipping /bucket/dist_haps/3L.02239835.02412740.npy
skipping /bucket/dist_haps/3L.02412741.02586161.npy
skipping /bucket/dist_haps/3L.02586162.02773590.npy
skipping /bucket/dist_haps/3L.02773591.02919374.npy
skipping /bucket/dist_haps/3L.02919375.03073996.npy
skipping /bucket/dist_haps/3L.03073997.03235327.npy
skipping /bucket/dist_haps/3L.03235328.03378695.npy
skipping /bucket/dist_haps/3L.03378696.03487521.npy
skipping /bucket/dist_haps/3L.03487522.03595558.npy
skipping /bucket/dist_haps/3L.03595559.03719733.npy
skipping /bucket/dist_haps/3L.03719734.03884028.npy
skipping /bucket/dist_haps/3L.03884029.04028601.npy
skipping /bu

In [56]:
#compute_dxy_distance_matrices('3R', 100000)

--------------------------------

## DXY plotting

In [4]:
palette = sns.color_palette()

In [5]:
distscan_dir = '/bucket/dist_haps/'
!ls -lh {distscan_dir}* | head

ls: cannot access '/bucket/dist_haps/*': No such file or directory


In [17]:
def open_dscan(chrom):

    # find the underlying matrix file names, we need these to get window boundaries
    dfns = sorted(glob.glob(os.path.join(distscan_dir, chrom + '*.npy')))
    bnms = [os.path.basename(f) for f in dfns]
    windows = np.array([[int(b.split('.')[1]), int(b.split('.')[2])] for b in bnms])
    n_windows = len(windows)
#     log(n_windows, 'windows')

    # compile into a single array
    dscan_rootdir = os.path.join(distscan_dir, '%s.bcolz' % chrom)
    if not os.path.exists(dscan_rootdir):
        log('loading', dscan_rootdir)

        # load up the first matrix to determine shape
        d = np.load(dfns[0])
        n_pairs = d.shape[0]
#         log(n_pairs, 'pairs')

        # setup bcolz array
        dscan = bcolz.carray(np.empty((0, n_pairs), dtype='f4'), 
                             cparams=bcolz.cparams(cname='zlib', clevel=1),
                             rootdir='/bucket/dist_haps/', #I have to change path beacuse gcs bucket I don't know why is bugged
                             mode='w',
                             expectedlen=n_windows)

        # load one row at a time
        for i, dfn in enumerate(dfns):
            if i > 0 and i % 20 == 0:
                log(i, dfn)
            dscan.append(np.load(dfn, mmap_mode='r'))
            dscan.flush()
        
    # open/re-open in read-only mode
    dscan = bcolz.carray(rootdir='/bucket/', mode='r')

    return windows, dscan

In [25]:
dfns = sorted(glob.glob(os.path.join('/bucket/dist_haps', '3L' + '*.npy')))
dfns

[]

In [18]:
open_dscan('3L')

loading /bucket/dist_haps/3L.bcolz


IndexError: list index out of range

In [20]:
def sample_to_haplotype_indices(sidx):
    return list(itertools.chain(*[[ix*2, ix*2+1] for ix in sidx])) 

In [21]:
def extract_2pop_dscan(chrom, pop1, pop2):

    callset = (calldata_hap_phase2)
    samples = tbl_samples
    n_samples = len(samples)
    n_haplotypes = 2 * n_samples
    
    # load dscan
    windows, dscan = open_dscan(chrom)

    # locate sample indices
    pop1_sample_indices=tbl_samples.population[tbl_samples.population == pop1].index.tolist()
    pop2_sample_indices=tbl_samples.population[tbl_samples.population == pop2].index.tolist()

    # locate haplotype indices
    pop1_haplotype_indices = sample_to_haplotype_indices(pop1_sample_indices)
    pop2_haplotype_indices = sample_to_haplotype_indices(pop2_sample_indices)
    
    # locate indices of pairwise comparisons within and between populations
    dw1_ix = allel.condensed_coords_within(pop1_haplotype_indices, n_haplotypes)
    dw2_ix = allel.condensed_coords_within(pop2_haplotype_indices, n_haplotypes)
    db_ix = allel.condensed_coords_between(pop1_haplotype_indices, pop2_haplotype_indices, n_haplotypes)
    
    # extract pairwise distances
    dw1 = allel.chunked.core.take(dscan, dw1_ix, axis=1)[:]
    dw2 = allel.chunked.core.take(dscan, dw2_ix, axis=1)[:]
    db = allel.chunked.core.take(dscan, db_ix, axis=1)[:]
    
    return windows, dw1, dw2, db


In [22]:
extract_2pop_dscan('3L', 'BFcol', 'BFgam')

loading /bucket/dist_haps/3L.bcolz
20 /bucket/dist_haps/3L.04210839.04880931.npy
40 /bucket/dist_haps/3L.08237501.08395362.npy
60 /bucket/dist_haps/3L.12103703.12259501.npy
80 /bucket/dist_haps/3L.15396153.15599366.npy
100 /bucket/dist_haps/3L.18602688.18743162.npy
120 /bucket/dist_haps/3L.21834103.22052592.npy
140 /bucket/dist_haps/3L.25376352.25594163.npy
160 /bucket/dist_haps/3L.28688683.28867194.npy
180 /bucket/dist_haps/3L.31776426.31940279.npy
200 /bucket/dist_haps/3L.34796002.34956052.npy
220 /bucket/dist_haps/3L.38108737.38268654.npy
240 /bucket/dist_haps/3L.40790864.40922676.npy


(array([[    9778,   730787],
        [  730788,  1243275],
        [ 1243276,  1622713],
        [ 1622714,  1833911],
        [ 1833912,  1989789],
        [ 1989790,  2108439],
        [ 2108440,  2239834],
        [ 2239835,  2412740],
        [ 2412741,  2586161],
        [ 2586162,  2773590],
        [ 2773591,  2919374],
        [ 2919375,  3073996],
        [ 3073997,  3235327],
        [ 3235328,  3378695],
        [ 3378696,  3487521],
        [ 3487522,  3595558],
        [ 3595559,  3719733],
        [ 3719734,  3884028],
        [ 3884029,  4028601],
        [ 4028602,  4210838],
        [ 4210839,  4880931],
        [ 4880932,  5102740],
        [ 5102741,  5235705],
        [ 5235706,  5365178],
        [ 5365179,  5483457],
        [ 5483458,  5625495],
        [ 5625496,  5737017],
        [ 5737018,  5881601],
        [ 5881602,  6017657],
        [ 6017658,  6216419],
        [ 6216420,  6382190],
        [ 6382191,  6520260],
        [ 6520261,  6644665],
        [ 

In [23]:
def plot_dist_scan(windows, d, ylim=(0, 0.016), title=None, color=None, ax=None):
    
    # set up figure
    if ax is None:
        fig, ax = subplots(figsize=(8, 1.5))
    if title is not None:
        ax.set_title(title, fontweight='bold')

    # window centres
    x = np.array(windows).mean(axis=1)

    # median
    y = np.median(d, axis=1)
    ax.plot(x, y, color=color, lw=1)
    
    # interquartile range
    y1 = np.percentile(d, 25, axis=1)
    y2 = np.percentile(d, 75, axis=1)
    ax.fill_between(x, y1, y2, color=color, alpha=.6)
    
    # 5-95 range
    y1 = np.percentile(d, 5, axis=1)
    y2 = np.percentile(d, 95, axis=1)
    ax.fill_between(x, y1, y2, color=color, alpha=.4)
    
    # total range
    y1 = d.min(axis=1)
    y2 = d.max(axis=1)
    ax.fill_between(x, y1, y2, color=color, alpha=.2)
    
    # tidy up
    ax.set_ylim(*ylim)
    
    # legend
    handles = list()
    # median
    l = plt.Line2D([], [], color=color, linestyle='-', linewidth=2, label='median')
    handles.append(l)
    # percentiles
    r = plt.Rectangle([0, 0], 1, 1, color=color, alpha=.6, lw=0, label='25-75th percentiles')
    handles.append(r)
    r = plt.Rectangle([0, 0], 1, 1, color=color, alpha=.4, lw=0, label='5-95th percentiles')
    handles.append(r)
    r = plt.Rectangle([0, 0], 1, 1, color=color, alpha=.2, lw=0, label='min-max')
    handles.append(r)
    ax.legend(handles=handles, bbox_to_anchor=[1, 1], loc='upper left')    
    ax.set_yticks([0, .005, .01, .015])
    ax.set_xlim(0, windows[-1, -1])
    

In [24]:
def plot_gmin(windows, d, title=None, ax=None):
    
    # set up figure
    if ax is None:
        fig, ax = subplots(figsize=(8, 1.5))
    if title is not None:
        ax.set_title(title, fontweight='bold')

    # window centres
    x = np.array(windows).mean(axis=1)

    # Gmin
    y = d.min(axis=1) / d.mean(axis=1)
    ax.plot(x, y, color='k', lw=1, linestyle='-')
        
    # tidy up
    ax.set_ylabel(r'$G_{min}$', rotation=0, ha='right', va='center')
    ax.set_xlim(0, windows[-1, -1])
    ax.set_ylim(0, 1)
    ax.set_yticks([0, 1])

In [25]:
autosomes = '3L'

In [26]:
def run_2pop_analysis(pop1, pop2):
    for chrom in autosomes:
        fig = plt.figure(figsize=(7, 6))
        windows, dw1, dw2, db = extract_2pop_dscan(chrom, pop1, pop2)
        
        ax = fig.add_subplot(4, 1, 1)
        sns.despine(ax=ax, bottom=True, offset=5)
        plot_dist_scan(windows, dw1, title='%s, %s' % (pop1, chrom), color=palette[0], ax=ax)
        ax.set_xticks([])
        ax.set_ylabel(r'$\pi$', rotation=0, ha='right', va='center')
        
        ax = fig.add_subplot(4, 1, 2)
        sns.despine(ax=ax, bottom=True, offset=5)
        plot_dist_scan(windows, dw2, title='%s, %s' % (pop2, chrom), color=palette[1], ax=ax)
        ax.set_xticks([])
        ax.set_ylabel(r'$\pi$', rotation=0, ha='right', va='center')
        
        ax = fig.add_subplot(4, 1, 3)
        sns.despine(ax=ax, bottom=True, offset=5)
        plot_dist_scan(windows, db, title='%s/%s, %s' % (pop1, pop2, chrom), color=palette[2], ax=ax)
        ax.set_xticks([])
        ax.set_ylabel(r'$d_{XY}$', rotation=0, ha='right', va='center')

        ax = fig.add_subplot(4, 1, 4)
        sns.despine(ax=ax, offset=5)
        plot_gmin(windows, db, title='%s/%s, %s' % (pop1, pop2, chrom), ax=ax)
        ax.set_xlabel('position (bp)')
        
        fig.tight_layout()
        plt.show()

In [32]:
run_2pop_analysis('GNgam', 'GNcol')

loading /bucket/dist_haps/3.bcolz


FileNotFoundError: [Errno 2] No such file or directory: '/bucket/dist_haps/bcolz/'

<Figure size 504x432 with 0 Axes>

In [ ]:
run_2pop_analysis('BFgam', 'BFcol')

In [ ]:
run_2pop_analysis('GHcol', 'GHgam')